In [ ]:
#%matplotlib inline
import numpy as np
import os
from tqdm import tqdm
from PIL import Image
from matplotlib.pyplot import imshow
import math
import sklearn

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from collections import namedtuple
from ase.io import read
from ase.build import molecule
from ase import Atoms, Atom
from ase.visualize import view
from ase.geometry.analysis import Analysis
import plotly.graph_objects as go
from ase.data import vdw_radii
from ase.data.colors import cpk_colors, jmol_colors

In [ ]:
from soap_generation.alignment import align_elements
from soap_generation.augment import augment_elements

In [ ]:
import csv

barriers = dict()

with open('../data/vaskas_features_properties_smiles_filenames.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        #images.append(row[0])
        #elos.append(row[1])
        barriers[row[93]] = float(row[91])

In [ ]:
labels = []
elems = []
for f in tqdm(os.listdir("../data/coordinates_molSimplify/")):
    if f.endswith(".xyz"):
        if (f == "ir_tbp_1_dft-pet3_1_dft-py_1_dft-hicn_1_fluoride_1_smi1_1_s_1.xyz"):
            print("1:" + str(len(labels)))

        if (f == "ir_tbp_1_dft-pet3_1_dft-py_1_dft-hicn_1_dft-cn_1_smi1_1_s_1.xyz"):
            print("2:" + str(len(labels)))
        elems.append(read("../data/coordinates_molSimplify/" + f))
        labels.append(barriers[f[:-4]])

labels = np.array(labels)
number_samples = len(labels)

In [ ]:
elems = align_elements(elems)

In [ ]:
species = ["H","C","N","O","F","P","S","Cl","As","Br","I","Ir"]

In [ ]:
import scipy.stats as st

def gkern(middle,kernlen=100, sigma=3):
    """Returns a 2D Gaussian kernel.""" 
    width = math.floor(kernlen / 2)
    x = np.arange(-width - middle[0],width - middle[0],1)
    y = np.arange(-width - middle[1],width - middle[1],1)
    z = np.arange(-width - middle[2],width - middle[2],1)
    xx, yy, zz = np.meshgrid(x,y,z)
    kernel = np.exp(-(xx**2 + yy**2 + zz**2)/(2*sigma**2))
    return kernel

In [ ]:
imshow(gkern([-33,0,-3])[0])

In [ ]:
element = elems[0]
voxel_rep = np.zeros((len(species),100,100,100))

SCALING = 5

for atom, location in zip(element.get_chemical_symbols(), element.get_positions()):
    atom_index = species.index(atom)
    voxel_rep[atom_index] += gkern(location*SCALING, sigma=8)

In [ ]:
imshow(voxel_rep[0][40])

In [ ]:
import plotly.graph_objects as go

import plotly.offline as pyo
pyo.init_notebook_mode()

X, Y, Z = np.mgrid[0:100:100j, 0:100:100j, 0:100:100j]
values = voxel_rep[0]

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=values.flatten(),
    isomin=0.1,
    isomax=0.8,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=17, # needs to be a large number for good volume rendering
    ))
fig.show()